In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Specify the file path in Google Drive
file_path = '/content/drive/MyDrive/sample/combined_data.csv'

# Read the CSV file into a DataFrame
combined_df = pd.read_csv(file_path)


In [3]:
num_rows = combined_df.shape[0]
print(f"Number of rows in the DataFrame: {num_rows}")


Number of rows in the DataFrame: 61262


In [ ]:
# reference - https://huggingface.co/distilroberta-base#model-details

In [4]:
# Assuming you have a DataFrame named 'combined_df'
# Replace 'combined_df' with the actual name of your DataFrame

# Calculate the number of DataFrames needed
num_samples = (len(combined_df) // 1000) + 1

# Create and store DataFrames
sample_dfs = []

for i in range(num_samples):
    start_index = i * 1000
    end_index = min((i + 1) * 1000, len(combined_df))
    sample_df = combined_df.iloc[start_index:end_index].copy()
    sample_dfs.append(sample_df)
    sample_df_name = f'sample_df{i + 1}'
    globals()[sample_df_name] = sample_df  # Create a variable with the DataFrame name

# Example: Access the first sample DataFrame
print(sample_df1)


             Publication Date           Source  \
0    2023-10-07T03:11:36+0000            Yahoo   
1    2023-10-07T03:42:10+0000            Yahoo   
2    2023-09-12T13:05:13+0000     Daily Caller   
3    2023-09-15T09:24:23+0000            Yahoo   
4    2023-09-14T01:26:43+0000    New York Post   
..                        ...              ...   
995  2023-03-16T16:16:28+0000        UrduPoint   
996  2023-05-02T12:00:56+0000        Live Mint   
997  2023-05-30T05:01:12+0000  Vietnam Express   
998  2023-05-12T16:28:48+0000             CNBC   
999  2023-05-05T17:52:23+0000          Reuters   

                                                   URL  \
0    https://news.yahoo.com/gov-renews-public-healt...   
1    https://news.yahoo.com/governor-extends-gun-vi...   
2    https://dailycaller.com/2023/09/12/poppy-harlo...   
3    https://news.yahoo.com/mexico-governors-gun-su...   
4    https://nypost.com/2023/09/13/new-mexico-judge...   
..                                                 

In [5]:
sample_df5.head()

,Publication Date,Source,URL,Title,Summary,Categories,Industries,Organisations,People,Locations,Other Entities,News topic
4000,2023-05-22T20:51:54+0000,Hk Finance Yahoo,https://hk.finance.yahoo.com/news/1-first-citi...,guce,(Adds HSBC's no comment in 4th paragraph and d...,Personal Finance\nStocks\nbanking\nBusiness an...,Financial Services and Banking\nBanks and Bank...,Silicon Valley Bank\nHSBC\nReuters\nFirst Citi...,NaN,United States,Daniel Wiessner,silicon valley bank collapse
4001,2023-05-01T08:58:05+0000,Yahoo! Finance News,https://finance.yahoo.com/news/regulators-clos...,"Regulators close First Republic Bank, JPMorgan...",Ingrid Lunden · 3 min read In this article:...,Personal Finance\nInsurance\nfinance (general)...,Financial Services and Banking\nBanks and Bank...,First Republic Bank\nJPMorgan Chase\nFederal D...,NaN,NaN,NaN,silicon valley bank collapse
4002,2023-04-14T10:02:51+0000,Market Watch,http://www.marketwatch.com/news/story.asp?guid...,: Charles Schwab top investor offloaded entire...,"One of Charles Schwab’s main investors, Florid...",Personal Finance\nStocks\nbanking\nFinance\nBa...,Financial Services and Banking\nBanks and Bank...,GQG Partners\nSilicon Valley Bank\nFinancial T...,NaN,Florida,Charles M. Schwab,silicon valley bank collapse
4003,2023-05-04T14:23:58+0000,Daily Mail UK,https://www.dailymail.co.uk/news/article-12046...,Nearly HALF of Americans are worried about the...,More people are worried about their deposits t...,Personal Finance\nStocks\nbanking\nSurveys and...,Financial Services and Banking\nBanks and Bank...,United States\nRepublican Party (United States...,NaN,United States,NaN,silicon valley bank collapse
4004,2023-03-13T19:52:06+0000,Medium,https://medium.com/@finfox/the-shocking-collap...,The Shocking Collapse of Silicon Valley Bank,What it Means for the Tech Industry and Your I...,Personal Finance\nBusiness\nBiotech/Biomedical...,Financial Services and Banking\nBanks and Bank...,Silicon Valley Bank\nSurinamese Football Assoc...,NaN,NaN,NaN,silicon valley bank collapse


In [10]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# Load pre-trained DistilRoBERTa model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Assuming your DataFrames are named 'sample_df1', 'sample_df2', ..., 'sample_df61'
# Replace 'sample_df1', 'sample_df2', etc. with the actual names of your DataFrames

# Specify the columns in your DataFrame for 'Title' and 'Summary'
title_column = 'Title'
summary_column = 'Summary'

# Set the sample number to start from
start_sample = 53

# Specify the number of samples
num_samples = 62

# Specify the directory to save CSV files
file_path = '/content/drive/MyDrive/sample/'

for sample_num in range(start_sample, num_samples + 1):
    # Construct the DataFrame name
    sample_df_name = f'sample_df{sample_num}'

    # Load the current sample DataFrame
    current_sample_df = globals()[sample_df_name]

    # Define a function for processing a batch of texts
    def process_batch(texts):
        results = []

        # Tokenize the entire batch
        tokenized = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**tokenized)

        # Transform logits to probabilities and get predicted label
        probs = torch.softmax(outputs.logits, dim=-1)
        preds = probs.argmax(dim=-1)
        labels = [model.config.id2label[pred.item()] for pred in preds]
        scores = probs.max(dim=-1).values.tolist()

        results.extend(zip(labels, scores))

        return results

    # Define a function to process the entire column in batches
    def process_column_in_batches(column):
        batch_size = 32  # You can adjust this based on your available memory
        results = []

        with tqdm(total=len(column), desc="Processing") as pbar:
            for i in range(0, len(column), batch_size):
                batch_texts = column[i:i + batch_size]
                batch_results = process_batch(batch_texts)
                results.extend(batch_results)
                pbar.update(len(batch_texts))

        return results

    # Process 'Title' column
    title_results = process_column_in_batches(current_sample_df[title_column].dropna().astype('str').tolist())

    # Process 'Summary' column
    summary_results = process_column_in_batches(current_sample_df[summary_column].dropna().astype('str').tolist())

    # Create new columns in the DataFrame for 'Title' predictions and scores
    current_sample_df['Title_Emotion'], current_sample_df['Title_Emotion_Score'] = zip(*title_results)

    # Create new columns in the DataFrame for 'Summary' predictions and scores
    current_sample_df['Summary_Emotion'], current_sample_df['Summary_Emotion_Score'] = zip(*summary_results)

    # Display the updated DataFrame
    print(current_sample_df[['Title', 'Title_Emotion', 'Title_Emotion_Score', 'Summary', 'Summary_Emotion', 'Summary_Emotion_Score']])

    # Save the current sample DataFrame to a CSV file in the specified directory
    current_sample_df.to_csv(f'{file_path}sample_results_{sample_num}.csv', index=False)

Processing: 100%|██████████| 999/999 [18:12<00:00,  1.09s/it]


                                                 Title Title_Emotion  \
0    Google to block local news in Canada in respon...         anger   
1    Supreme Court rules environmental impact legis...         anger   
2    Meta starts blocking the news in Canada — its ...         anger   
3    Google, Meta Take on Trudeau in Proxy Fight Ov...       neutral   
4    Canada and Whitecap Dakota Nation mark Royal A...       neutral   
..                                                 ...           ...   
994  UK government blocks Scotland's gender reform ...         anger   
995  FM Bilawal to attend Shanghai Cooperation Orga...       neutral   
996  Three years after Galwan, disengagement has no...       neutral   
997     Kashmir issue on Pakistan's agenda in SCO meet          fear   
998  Police Scotland divers recover bodies of two m...          fear   

     Title_Emotion_Score                                            Summary  \
0               0.759921  Google on Thursday became the 

Processing:  48%|████▊     | 480/1000 [00:30<00:32, 15.93it/s]


KeyboardInterrupt: ignored

In [8]:
# Assuming 'sample_df53' is your DataFrame
empty_summary_rows = sample_df53[sample_df53['Summary'].isnull() | (sample_df53['Summary'] == '')]

# Print the rows with empty 'Summary'
print("Rows with Empty 'Summary':")
print(empty_summary_rows)


Rows with Empty 'Summary':
               Publication Date  Source  \
52956  2021-08-26T08:06:54+0000  BBC UK   

                                                     URL  \
52956  https://www.bbc.co.uk/news/uk-scotland-scotlan...   

                                                   Title Summary  \
52956  Scottish Greens co-leaders Harvie and Slater t...     NaN   

                                              Categories Industries  \
52956  U.S. Government Resources\nLaw, Gov’t & Politi...        NaN   

                  Organisations  People                 Locations  \
52956  United Kingdom\nScotland     NaN  United Kingdom\nScotland   

                     Other Entities                      News topic  
52956  Patrick Harvie\nBilly Slater  operation branchform, scotland  


In [9]:
# Drop rows with empty 'Summary'
sample_df53 = sample_df53.dropna(subset=['Summary'])

# Reset the index after dropping rows
sample_df53 = sample_df53.reset_index(drop=True)


In [6]:
# Assuming 'current_sample_df' is your DataFrame
print("DataFrame Index:")
print(sample_df53.index)


DataFrame Index:
RangeIndex(start=52000, stop=53000, step=1)


In [11]:
import pandas as pd
import os

# Specify the directory where your CSV files are located
directory = '/content/drive/MyDrive/sample/'

# List to store DataFrames
dfs = []

# Loop through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv') and filename.startswith('sample_results_'):
        file_path = os.path.join(directory, filename)
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('/content/drive/MyDrive/sample/combined_results.csv', index=False)

# Display the combined DataFrame
print(combined_df)


               Publication Date                Source  \
0      2023-10-07T03:11:36+0000                 Yahoo   
1      2023-10-07T03:42:10+0000                 Yahoo   
2      2023-09-12T13:05:13+0000          Daily Caller   
3      2023-09-15T09:24:23+0000                 Yahoo   
4      2023-09-14T01:26:43+0000         New York Post   
...                         ...                   ...   
61256  2023-01-17T05:34:38+0000  Yahoo UK and Ireland   
61257  2023-04-20T07:53:05+0000                  Dawn   
61258  2023-06-19T02:40:16+0000        Indian Express   
61259  2023-05-03T10:54:17+0000             Msn India   
61260  2023-02-25T16:51:26+0000          The Guardian   

                                                     URL  \
0      https://news.yahoo.com/gov-renews-public-healt...   
1      https://news.yahoo.com/governor-extends-gun-vi...   
2      https://dailycaller.com/2023/09/12/poppy-harlo...   
3      https://news.yahoo.com/mexico-governors-gun-su...   
4      https://